## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,13.34,87,100,5.66,overcast clouds
1,1,Hermanus,ZA,-34.4187,19.2345,15.53,86,7,4.19,clear sky
2,2,Carnarvon,AU,-24.8667,113.6333,20.04,88,40,3.09,scattered clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,-24.47,77,3,0.51,clear sky
4,4,Isangel,VU,-19.5500,169.2667,26.48,81,100,1.12,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 30


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,13.34,87,100,5.66,overcast clouds
1,1,Hermanus,ZA,-34.4187,19.2345,15.53,86,7,4.19,clear sky
2,2,Carnarvon,AU,-24.8667,113.6333,20.04,88,40,3.09,scattered clouds
4,4,Isangel,VU,-19.5500,169.2667,26.48,81,100,1.12,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,25.74,70,1,5.76,clear sky
7,7,Paka,MY,4.6374,103.4368,25.96,94,94,0.00,overcast clouds
9,9,Mattru,SL,7.6244,-11.8332,26.28,72,100,2.97,overcast clouds
11,11,Port Blair,IN,11.6667,92.7500,27.59,73,58,0.76,broken clouds
15,15,Dubrovnik,HR,42.6481,18.0922,12.80,32,0,14.40,clear sky
18,18,Tautira,PF,-17.7333,-149.1500,27.26,73,18,5.96,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                449
City                   449
Country                443
Lat                    449
Lng                    449
Max Temp               449
Humidity               449
Cloudiness             449
Wind Speed             449
Current Description    449
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

clean_preferred_cities_df.count()


City_ID                443
City                   443
Country                443
Lat                    443
Lng                    443
Max Temp               443
Humidity               443
Cloudiness             443
Wind Speed             443
Current Description    443
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,13.34,overcast clouds,-42.8794,147.3294,
1,Hermanus,ZA,15.53,clear sky,-34.4187,19.2345,
2,Carnarvon,AU,20.04,scattered clouds,-24.8667,113.6333,
4,Isangel,VU,26.48,overcast clouds,-19.5500,169.2667,
5,Rikitea,PF,25.74,clear sky,-23.1203,-134.9692,
7,Paka,MY,25.96,overcast clouds,4.6374,103.4368,
9,Mattru,SL,26.28,overcast clouds,7.6244,-11.8332,
11,Port Blair,IN,27.59,broken clouds,11.6667,92.7500,
15,Dubrovnik,HR,12.80,clear sky,42.6481,18.0922,
18,Tautira,PF,27.26,light rain,-17.7333,-149.1500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude, longitude, city and country from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    city=row['City']
    country=row['Country']

    params["location"] = f"{lat}, {lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found in " + city + ", " + country + "...skipping.")

Hotel not found in Abu Samrah, SY...skipping.
Hotel not found in Aitape, PG...skipping.
Hotel not found in Poum, NC...skipping.
Hotel not found in Kisesa, TZ...skipping.
Hotel not found in Kamenka, RU...skipping.
Hotel not found in Samoylovka, RU...skipping.
Hotel not found in Qandala, SO...skipping.
Hotel not found in Aguas Vermelhas, BR...skipping.
Hotel not found in Mahanje, TZ...skipping.
Hotel not found in Carnot, CF...skipping.
Hotel not found in Uruzgan, AF...skipping.
Hotel not found in Kovalevskoye, RU...skipping.
Hotel not found in Gazli, UZ...skipping.
Hotel not found in Yaan, NG...skipping.
Hotel not found in Nioro, GM...skipping.
Hotel not found in Avera, US...skipping.
Hotel not found in Airai, TL...skipping.
Hotel not found in Adrar, MR...skipping.
Hotel not found in Tessalit, ML...skipping.
Hotel not found in Maridi, SS...skipping.
Hotel not found in Tsaratanana, MG...skipping.
Hotel not found in Ndele, CF...skipping.
Hotel not found in Marzuq, YE...skipping.
Hotel not 

In [9]:
# 7. Drop the rows where there is no Hotel Name.
# 7a. Determine if there are any empty rows.
hotel_df.count()

City                   443
Country                443
Max Temp               443
Current Description    443
Lat                    443
Lng                    443
Hotel Name             443
dtype: int64

In [10]:
# 7b. Replace blank rows to NaN and then drop NaN to create a new Dataframe.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.count()

City                   407
Country                407
Max Temp               407
Current Description    407
Lat                    407
Lng                    407
Hotel Name             407
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))